In [1]:
%load_ext autoreload
%autoreload 2

# Misc

In [2]:
from synbio_morpher.utils.misc.type_handling import flatten_listlike
import numpy as np

In [39]:

arr = np.arange(6) * 2
# def make_symmetrical_matrix_from_sequence(arr, side_length: int):
side_length = 3

if len(arr) < (side_length ** 2):
    p = flatten_listlike([[str(tuple(sorted([i, j]))) for i in range(side_length)] for j in range(side_length)])
    # d = {pi: i for i, pi in enumerate(set(p))}
    d = np.concatenate([np.where(np.array(sorted(set(p))) == pi)[0] for pi in p]).astype(int)
    arr = arr[d]
i = np.triu(
    np.arange(side_length * side_length).reshape(side_length, side_length))
# idxs = np.interp(ii, np.unique(ii), np.arange(len(arr))).astype(int)
ii = i + i.T - np.diag(i.diagonal())
m = arr[ii.flatten()].reshape(side_length, side_length)


# m = make_symmetrical_matrix_from_sequence(np.arange(6), side_length=3)


In [40]:
m

array([[ 0,  2,  4],
       [ 2,  6,  8],
       [ 4,  8, 10]])

In [35]:
arr[d]

array([], shape=(9, 1, 0), dtype=int64)

In [29]:
np.array(sorted(set(p)))

array(['(0, 0)', '(0, 1)', '(0, 2)', '(1, 1)', '(1, 2)', '(2, 2)'],
      dtype='<U6')